# Libraries

In [44]:
# Data manipulation libraries
import pandas as pd # Dataframes
import numpy as np # Numerical operations

# Visualization libraries
import plotly.graph_objects as go # Interactive visualizations
import plotly.express as px # Simplified plotting with Plotly
from plotly.subplots import make_subplots # Complex subplot layouts

# Statistical libraries
import scipy.stats as stats # Statistical tests and distributions

# Dataset description

In [27]:
df = pd.read_csv(
    'data/food_delivery_times.csv',

    # Order ID treated as string to exclude from numerical analysis. 
    dtype = {'Order_ID': str}
) 
df.head()

,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68


In [28]:
df.dtypes

Order_ID                   object
Distance_km               float64
Weather                    object
Traffic_Level              object
Time_of_Day                object
Vehicle_Type               object
Preparation_Time_min        int64
Courier_Experience_yrs    float64
Delivery_Time_min           int64
dtype: object

In [29]:
df.describe()

,Distance_km,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
count,1000.000000,1000.000000,970.000000,1000.000000
mean,10.059970,16.982000,4.579381,56.732000
std,5.696656,7.204553,2.914394,22.070915
min,0.590000,5.000000,0.000000,8.000000
25%,5.105000,11.000000,2.000000,41.000000
50%,10.190000,17.000000,5.000000,55.500000
75%,15.017500,23.000000,7.000000,71.000000
max,19.990000,29.000000,9.000000,153.000000


# Numeric variables

In [30]:
num_df = df.select_dtypes(include = [np.number])
num_df.head()

,Distance_km,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,7.93,12,1.0,43
1,16.42,20,2.0,84
2,9.52,28,1.0,59
3,7.44,5,1.0,37
4,19.03,16,5.0,68


In [31]:
fig_heatmap = px.imshow(
    img = num_df.corr(),
    title = 'Correlation Heatmap of Numerical Features',
    height = 500,
    width = 600,
    template = 'plotly_dark',
    color_continuous_scale = 'Agsunset',
    text_auto = '.2f',
)
fig_heatmap.show()

In [32]:
fig_pairplot = px.scatter_matrix(
    data_frame = num_df,
    title = 'Scatter Matrix of Numerical Features',
    height = 900,
    template = 'plotly_dark',
    color_discrete_sequence = px.colors.qualitative.Safe,
)
fig_pairplot.show()

In [33]:
fig_dist = make_subplots(
    rows = num_df.shape[1] // 2,
    cols = 2,
    subplot_titles = num_df.columns 
).update_layout(
    title_text = "Distribution of Numerical Features",
    showlegend = False,
    xaxis={'categoryorder': 'category ascending'},
    template = 'plotly_dark',
    height = 600,
)

for i in range(num_df.shape[1]):
    fig_dist.add_trace(
        go.Histogram(
            x = num_df.iloc[:, i],
            name = num_df.columns[i],
            marker_color = px.colors.qualitative.Safe[i]
        ),
        row = (i // 2) + 1,
        col = (i % 2) + 1,
    )

fig_dist.show()

In [52]:
def kolmogorov_smirnov_test(data, dist_name, alpha = 0.05, verbose = False):
    dist = getattr(stats, dist_name)
    param = dist.fit(data)

    D, p = stats.kstest(data, dist_name, args = param)
    if verbose:
        print(f'p-value for {dist_name} distribution = ' + str(p))
        if p < alpha:
            print(f'Reject null hypothesis at alpha = {alpha}. Data does not follow {dist_name} distribution.')
        else:
            print(f'Data follows {dist_name} distribution.')
    return p

In [66]:
for i in range(num_df.shape[1]):
    print(f'Feature: {num_df.columns[i]}')

    for dist_name in ['norm', 'uniform', 'gamma']:
        kolmogorov_smirnov_test(num_df.iloc[:, i].dropna(), dist_name, verbose = True)
    print('---')

Feature: Distance_km
p-value for norm distribution = 0.0002072010235332123
Reject null hypothesis at alpha = 0.05. Data does not follow norm distribution.
p-value for uniform distribution = 0.20717998989270037
Data follows uniform distribution.
p-value for gamma distribution = 0.00020081517047715872
Reject null hypothesis at alpha = 0.05. Data does not follow gamma distribution.
---
Feature: Preparation_Time_min
p-value for norm distribution = 2.480592229228519e-07
Reject null hypothesis at alpha = 0.05. Data does not follow norm distribution.
p-value for uniform distribution = 0.05706939112023113
Data follows uniform distribution.
p-value for gamma distribution = 2.994200185671417e-07
Reject null hypothesis at alpha = 0.05. Data does not follow gamma distribution.
---
Feature: Courier_Experience_yrs
p-value for norm distribution = 2.8353579015889568e-12
Reject null hypothesis at alpha = 0.05. Data does not follow norm distribution.
p-value for uniform distribution = 6.274185660160615e

In [34]:
fig_box = make_subplots(
    rows = 1, 
    cols = num_df.shape[1],
).update_layout(
    title_text = "Box plots of Numerical Features",
    showlegend = False,
    template = 'plotly_dark'
)

for i in range(num_df.shape[1]):
    fig_box.add_trace(
        go.Box(
            y = num_df.iloc[:,i],
            name = num_df.columns[i],
            marker_color = px.colors.qualitative.Safe[i]
        ),
        row = 1, 
        col = i+1,
    )

fig_box.show()

# Export figures

In [35]:
fig_heatmap.write_image('figures/heatmap_num.png', scale = 2)
fig_pairplot.write_image('figures/pairplot_num.png', scale = 2)
fig_dist.write_image('figures/distribution_num.png', scale = 2)
fig_box.write_image('figures/boxplots_num.png', scale = 2)